# WTMetad simulation with deep TICA bias potential

## Input
* Starting structure (pdb)
* `net`: inference network with TICA layer from `SRV.srv_net(num_cvs)`

## Output
* mdtraj trajectory files in `data/ala2_metad` or `data/ala2_metad_vac`
* `Metadynamics` object for each simulaiton, before and after KDE compression

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import mdtraj as md 
import openmm as mm
import openmm.unit as unit
from openmm.app import Simulation, ForceField, HBonds
from openmmtorch import TorchForce
import os

from src.filenames import FileNames
from src.util import SimulationParametersMetaD, SimulationCVS, pretty_print
from src.util import save_npz, load_npz, write_pdb
from src.util import create_system, state_data_reporter, hdf5_reporter
from src.metad import metadynamics, kde_compression

if torch.cuda.is_available():
    device = torch.device('cuda')
    torch.backends.cudnn.benchmark = True
else:
    device = torch.device('cpu')
    torch.set_num_threads(12)
%config Completer.use_jedi = False
print(f'{device = }, {torch.backends.cudnn.enabled = }')
print(f'{torch.cuda.is_available() = }, {torch.__version__ = }')

device = device(type='cuda'), torch.backends.cudnn.enabled = True
torch.cuda.is_available() = True, torch.__version__ = '1.9.0'


# Parameters

**References**

Parrinello (2008): "Well-Tempered Metadynamics: A Smoothly Converging and Tunable Free-Energy Method" Alessandro Barducci, Giovanni Bussi, and Michele Parrinello, _Phys. Rev. Lett._, **100**, 020603 (2008) https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.100.020603

Bussi (2015):  "Free-Energy Calculations with Metadynamics: Theory and Practice", Giovanni Bussi and Davide Branduardi, Reviews in Computational Chemistry Volume 28, 2015 https://doi.org/10.1002/9781118889886.ch1

Vandrasek (2010): "Metadynamics As a Tool for Mapping the Conformational and Free-Energy Space of Peptides — The Alanine Dipeptide Case Study", Jiří Vymětal and Jiří Vondrášek, _J. Phys. Chem. B_ , **114**, 16, 5632 (2010) https://pubs.acs.org/doi/abs/10.1021/jp100950w


**Bias factor**

Parrinello (2008): $T = 300$ K, $\Delta T = 1200$ K optimal wrt CLT:
$$
\lim_{t\rightarrow\infty}\frac{\langle\epsilon(t)\rangle}{\sqrt{t}}
$$

$$
\gamma = \frac{\beta}{\beta'} + 1 = \frac{\Delta T}{T} + 1 = \frac{1200 K}{300 K} + 1 = 5
$$

**Gaussian heights, $\tau_G$, deposition rate**

Bussi (2015): "A typical choice is on the order of 0.1 kcal/mol (0.418 kJ/mol) for systems simulated at room temperature."  Choose $\tau_G$ to be on the order of the autocorrelation time of the CV.  

I'm using Parrinello (2008) $h$ = 0.287 kcal/mol = 1.20 kJ/mol.

$$
h = \omega e^{-w(s, t)/\Delta T}\tau_G ~~\rightarrow~~ w(s, 0) = 0 
~~\rightarrow~~ h = 1.20 \textrm{ kJ/mol}
$$

| Paper      | $\tau_G$ | $h$ | $\omega = h/\tau_G$ |
| ----------- | ----------- | ----- | ---- |
| Parrinello, 2008      | 120 fs | 0.288 kcal/mol | 2.4 cal/mol/fs |
|       | 0.12 ps | 1.20 kJ/mol | 10.0 J/mol/fs |
| Vandrasek, 2010      | 4000 fs |   |   |
|       | 4 ps | 0.2, 0.1, 0.05 kJ/mol | 1.7, 0.8, 0.4 J/mol/fs |
| Bussi, 2015      | 1200 fs | 0.1 kcal/mol |  0.08 cal/mol/fs |
|       | 1.2 ps | 0.418 kJ/mol | 0.35 J/mol/fs |

**Gaussian Widths**

Bussi (2015): good choice ~ 1/2 or 1/3 width of peak of $p(s)$.  

| Paper      | $\sigma_G$ | Fraction of p(s) Gaussian width |
| ----------- | ----------- | ----- |
| Parrinello, 2008      | $20^\circ$ | > 1 |
| Vandrasek, 2010      | $17^\circ$ | > 1 |
| Bussi, 2015      | $5.7^\circ$ | 1/2 |

Using histograms of $s_1$, $s_2$ over mdshare data and the formula relating 2D Gaussian height and width:
$$
\sigma = \frac{1}{\sqrt{2\pi h}}
$$
we get $h = 90$ (see hist2d) and $\sigma = 0.04$ is an upper bound (since we've assumed a Gaussian shape).  But we don't really care about a finely detailed $F(s)$ and we need to cover huge ranges of $s$, so $\sigma = 0.1$ is ok?

# Simulation

Simulations with `sim_num` > 0:
* have a starting configuration equal to the final configuration of the previous simulation
* start with the compressed set of Gaussian kernels from the previous simulation

In [2]:
p = SimulationParametersMetaD(
    simulation_time = 0.1 * unit.nanosecond, 
    report_interval_time = 1 * unit.picosecond,
    num_simulations = 300
)
# Figure out how gentle WTMetaD has to be to prevent blowup
# p.tau_G = 1 * unit.picosecond
# p.height = 0.01 * unit.kilojoule_per_mole
pretty_print(p)

simulation_time = 0.1 ns
report_interval_time = 1 ps
num_simulations = 300
temperature = 300 K
timestep = 0.002 ps
friction_coeff = 1 /ps
report_interval = 500
total_simulation_time = 30.0 ns
ns = 0.1
total_ns = 30.0
tau_G = 120 fs
height = 1.2 kJ/mol
width = [0.1 0.1]
bias_factor = 5.0
num_gaussians = 833
steps_per_gaussian = 60


In [3]:
solvated = True
if solvated:
    scv = SimulationCVS('data/ala2_solv.pdb')
    fn = FileNames('data/ala2_metad', p.ns)
    net = torch.load('data/srv.pt').eval()
    forcefield = ForceField('amber99sb.xml', 'tip3p.xml')
else:
    scv = SimulationCVS('data/ala2.pdb')
    fn = FileNames('data/ala2_metad_vac', p.ns)
    net = torch.load('data/srv.pt').eval() # TODO: train network for vacuum
    forcefield = ForceField('amber99sb.xml')
pdb = scv.pdb()
    
sim_start = 0
for sim_num in range(sim_start, p.num_simulations):
    print(f'starting simulation {sim_num}')
    if sim_num == 0:
        metad = metadynamics(p.temperature, p.bias_factor, p.height, p.width)
        metad.add_gaussian(scv.pdb_cvs(net))
        start_positions = pdb.positions
    else:
        metad = load_npz(fn.metad_kde(sim_num - 1))
        start_positions = load_npz(fn.final_positions(sim_num - 1))
        
    fm = metad.force_module(net)
    torch.jit.script(fm).save('data/force_module.pt')
    torch_force = TorchForce('data/force_module.pt')
    torch_force.addGlobalParameter('add_gaussian', False)
    torch_force.addGlobalParameter('height', 0.0)
    torch_force.addGlobalParameter('center1', 0.0)
    torch_force.addGlobalParameter('center2', 0.0)

    integrator = mm.LangevinIntegrator(p.temperature, p.friction_coeff, p.timestep)
    if solvated:
        system = create_system(forcefield, pdb.topology)
    else:
        system = forcefield.createSystem(pdb.topology, constraints=HBonds)
    system.addForce(torch_force)
    
    simulation = Simulation(pdb.topology, system, integrator)
    simulation.context.setPositions(start_positions)
    sdr = state_data_reporter(fn.out(sim_num), p.report_interval)
    hdr = hdf5_reporter(fn.h5(sim_num), p.report_interval)
    simulation.reporters.append(sdr)
    simulation.reporters.append(hdr)
    simulation.step(p.steps_per_gaussian)

    for n in range(p.num_gaussians - 1):
        metad.add_gaussian(scv.sim_cvs(net, simulation))
        simulation.context.setParameter('add_gaussian', True)
        simulation.context.setParameter('height', metad.heights[-1])
        simulation.context.setParameter('center1', metad.centers[-1][0])
        simulation.context.setParameter('center2', metad.centers[-1][1])
        simulation.step(1)
        simulation.context.setParameter('add_gaussian', False)
        simulation.step(p.steps_per_gaussian - 1)

    r = simulation.context.getState(getPositions=True).getPositions()
    save_npz(fn.final_positions(sim_num), r)
    save_npz(fn.metad(sim_num), metad)
    metad_kde = kde_compression(metad, dist_threshold=1)
    save_npz(fn.metad_kde(sim_num), metad_kde)
    print(f'compressing metad: {len(metad)} -> {len(metad_kde)} kernels\n')
    for reporter in simulation.reporters:
        if hasattr(reporter, 'close'):
            reporter.close()

starting simulation 0
compressing metad: 833 -> 13 kernels

starting simulation 1
compressing metad: 845 -> 193 kernels

starting simulation 2
compressing metad: 1025 -> 265 kernels

starting simulation 3
compressing metad: 1097 -> 282 kernels

starting simulation 4
compressing metad: 1114 -> 285 kernels

starting simulation 5
compressing metad: 1117 -> 304 kernels

starting simulation 6
compressing metad: 1136 -> 335 kernels

starting simulation 7
compressing metad: 1167 -> 367 kernels

starting simulation 8
compressing metad: 1199 -> 381 kernels

starting simulation 9
compressing metad: 1213 -> 381 kernels

starting simulation 10
compressing metad: 1213 -> 381 kernels

starting simulation 11
compressing metad: 1213 -> 383 kernels

starting simulation 12
compressing metad: 1215 -> 383 kernels

starting simulation 13
compressing metad: 1215 -> 383 kernels

starting simulation 14
compressing metad: 1215 -> 384 kernels

starting simulation 15
compressing metad: 1216 -> 404 kernels

start

KeyboardInterrupt: 

Closing remaining open files:data/ala2_metad/ala2_0.1ns_100.h5...done
